In [1]:
# Needed libraries
#
import numpy as np
from numba import jit, njit, prange

#numba.config.DEBUG_ARRAY_OPT=1
#numba.config.NUMBA_PARALLEL_DIAGNOSTICS=1


#numba.config.NUMBA_DISABLE_JIT=True
import time
import importlib
import pickle

import CLEDB_utils_parallel as cle202_utils   # Utilities to help the inversion

import os

#s.environ["NUMBA_DISABLE_JIT"] = "1"

<Figure size 1080x1080 with 0 Axes>

## 3 types of simulated/faked observations

In [2]:
## use fake_obs to just generate a random map.
# FE XIII 1074+1079
## This has the benefit of also testing the uncertainty implementation
## THIS IS VERY HACKISH!

# with open('obsstokes_dbfake.pkl','rb') as f:
#     sobs_a,yobs_a,rms,xxl,yyl = pickle.load(f) 


In [3]:
## load the CLE 3dipole fake observation data
## FE XIII 1074+1079
## sobs_a is normalized to first column (stokes I 1074)
## RMS is 1 in this example; observation can be convoluted with noise if needed and re-normalized again

with open('obsstokes_3dipole_hires.pkl','rb') as f:
    sobs_a,yobs_a,rms,xxl,yyl = pickle.load(f) 

In [4]:
## load the fake observation muram data.
## FE XIII 1074+1079
## sobs_a is normalized to first column (stokes I 1074)
## RMS is 1 in this example; observation can be convoluted with noise if needed and re-normalized again

# with open('obsstokes_avg_muram.pkl','rb') as f:
#     f1aa,f2aa,sobs_a,yobs_a,rms,wvl,xxl,yyl = pickle.load(f)    

# with open('obsstokes_avg_muram2.pkl','rb') as f:
#     sobs_a,yobs_a,rms,xxl,yyl = pickle.load(f)        

# DBINVERT MAIN!

In [5]:
################This is the DBINVERT script#################### 
importlib.reload(cle202_utils)

##Global constants & main controling parameters:

reduce = np.int_(1)  ## Uses native simmetries to reduce the database search; really fast only if enabled.
chi2   = np.int_(50) ## the maximum chi^2 to return; ideally a normalized and reduced chi^2 should be <1
ns     = np.int_(12)  ##maximum number of solutions to return

## Twoline databases are implemented; individual ion databases are partially implemented (a switcher to select what combination is requested is missing).
dbdir  = '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_deg_150_fe13/' ## main directory containing the two line Fe XIII DB entries; 
#dbdir  = '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_R0500/' ## main directory containing the individual ion DB entries; 

verbose= np.int_(3)  ## incremental verbosity of inversion operations
##not fully implememted
#verbose=1 gives prints for current operation being done
#verbose=2 implements warnings for common caveats.
#Verbose=3 will enable execution timing for sections that are not numba non-python enabled.

out=np.zeros((xxl,yyl,ns,11),dtype=np.float64)     
iquv_match=np.empty((xxl,yyl,ns,8),dtype=np.float64)
#Arrays to store the DBINVERT solutions. Provisioned to store the maximum amount of solutions retrievable(ns). 
#The inversion can produce less than ns solution due to the chi2 condition.
##out[0],[1] are the DB index and chi2 fit values respectively.
##out[2],[3],[4] are the plasma density, radial height position (y), and los position (x)
##out[5],[6],[7] are the LVS field B (scaled using the V_obs/V_db ratio) with phi and theta angles
##out[8],[9],[10] are bx by bz scaled using the V_obs/V_db ratio

if verbose >=1:
    print('####################################\n##########DBINVERT STARTED##########\n####################################')
    if verbose >=3: start=time.time()

# fenc,db1,hdrparams=cle202_utils.db_find(yobs_a,
#                                              dbdir1,
#                                              xxl,
#                                              yyl,
#                                              verbose)  
fenc,db,hdrparams=cle202_utils.db_find(yobs_a,
                                             dbdir,
                                             xxl,
                                             yyl,
                                             verbose)  

####################################
##########DBINVERT STARTED##########
####################################
------------DB READ START-----------
DB covers a span of  11  heights between 1.001 - 1.49 Solar radius
Load  11  DB heights in memory
----------
READING INDIVIDUAL DATA...
DB file location: /home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_deg_150_fe13/DB0450.DAT
155  MB in DB file
0.614798  SECONDS FOR INDIVIDUAL DB READ
----------
(10, 60, 180, 90, 8)
READING INDIVIDUAL DATA...
DB file location: /home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_deg_150_fe13/DB0490.DAT
155  MB in DB file
0.604486  SECONDS FOR INDIVIDUAL DB READ
----------
(10, 60, 180, 90, 8)
READING INDIVIDUAL DATA...
DB file location: /home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_deg_150_fe13/DB0301.DAT
155  MB in DB file
0.596060  SECONDS FOR INDIVIDUAL DB READ
----------
(10, 60, 180, 90, 8)
READING INDIVIDUAL DATA...
DB file location: /home/alin/Documents/ph

In [11]:
importlib.reload(cle202_utils)
os.environ["NUMBA_DISABLE_JIT"] = "0"

if verbose >=1: print('------------TWO LINE INVERSION START-----------')
@njit(parallel=True)
def dbinvert_twoline_par(sobs,database,fe,yobs,dbdir,rms,hdrparams,xxl,yyl,nsearch,chi2,reduced,verbose):
    np.argpartition(sobs,4)
    ##Output variables
    outparams=np.zeros((xxl,yyl,nsearch,11),dtype=np.float64)     
    smatch  =np.zeros((xxl,yyl,nsearch,8),dtype=np.float64)  
    
    ## Number of degrees of freedom >4; we require a two line full IQUV observation.
    ## assume the last dimension of sobs is the number of observables; 2 lines=8
    ## Database needs to match the observation
    if sobs.shape[-1] != 8 or database[0].shape[-1] != 8: 
        print('clematch: Must have a two line observation and/or a corresponding two line database; Aborting!')
        if verbose >=1: print("Shapes are: ", sobs.shape," and ",database[0].shape)
        outparams[:,:,:,0]=np.full((xxl,yyl,nsearch),-1) ### index is -1 for failed runs!
        return outparams,smatch
    
    
#     index   = np.empty((xxl,yyl,nsearch),dtype=np.int_)
#     chisq   = np.empty((xxl,yyl,nsearch),dtype=np.float64)
#     physics = np.empty((xxl,yyl,nsearch,9),dtype=np.float64)

    if verbose >= 1:      
        for xx in range(xxl):
            print("Executing ext. loop:",xx," of ",xxl," (",yyl," calculations / loop)")
            for yy in prange(yyl):
#                 index[xx,yy,:],smatch[xx,yy,:,:],chisq[xx,yy,:],physics[xx,yy,:,:]=cle202_utils.clematch_par(sobs[xx,yy,:],
#                                                                                                              yobs[xx,yy],
#                                                                                                              database[fe[xx,yy]],
#                                                                                                              hdrparams,
#                                                                                                              rms,
#                                                                                                              '',
#                                                                                                              chi2,
#                                                                                                              nsearch,
#                                                                                                              reduced,
#                                                                                                              0)

                outparams[xx,yy,:,:],smatch[xx,yy,:,:]=cle202_utils.clematch_par(sobs[xx,yy,:],yobs[xx,yy],database[fe[xx,yy]],hdrparams, rms, '',chi2,nsearch, reduced, 0)
          
            
            #outparams[xx,yy,:,0]=index[xx,yy,:]
            #outparams[xx,yy,:,1]=chisq[xx,yy,:]
            #outparams[xx,yy,:,2:11]=physics[xx,yy,:,:]
#     else:
#         for xx in range(xxl):
#             for yy in prange(yyl):

#                 index[xx,yy,:],smatch[xx,yy,:,:],chisq[xx,yy,:],physics[xx,yy,:,:]=cle202_utils.clematch_par(sobs[xx,yy,:],
#                                                                                                              yobs[xx,yy],
#                                                                                                              database[fe[xx,yy]],
#                                                                                                              hdrparams,
#                                                                                                              rms,                                         '',
#                                                                                                              chi2,
#                                                                                                              nsearch,
#                                                                                                              reduced,
#                                                                                                              verbose)        
               
#     outparams[:,:,:,0]=index
#     outparams[:,:,:,1]=chisq
#     outparams[:,:,:,2:11]=physics

    return outparams,smatch

out,iquv_match=dbinvert_twoline_par(sobs_a,db,fenc,yobs_a,dbdir,rms,hdrparams,xxl,yyl,ns,chi2,reduce,verbose)


if verbose >=1:
    print('####################################\n##########DBINVERT FINALIZED##########\n####################################')
    if verbose >=3: print("{:4.6f}".format(time.time()-start),' TOTAL SECONDS FOR DBINVERT ')

------------TWO LINE INVERSION START-----------
Executing ext. loop: 0  of  280  ( 600  calculations / loop)
Executing ext. loop: 1  of  280  ( 600  calculations / loop)
Executing ext. loop: 2  of  280  ( 600  calculations / loop)
Executing ext. loop: 3  of  280  ( 600  calculations / loop)
Executing ext. loop: 4  of  280  ( 600  calculations / loop)
Executing ext. loop: 5  of  280  ( 600  calculations / loop)
Executing ext. loop: 6  of  280  ( 600  calculations / loop)
Executing ext. loop: 7  of  280  ( 600  calculations / loop)
Executing ext. loop: 8  of  280  ( 600  calculations / loop)
Executing ext. loop: 9  of  280  ( 600  calculations / loop)
Executing ext. loop: 10  of  280  ( 600  calculations / loop)
Executing ext. loop: 11  of  280  ( 600  calculations / loop)
Executing ext. loop: 12  of  280  ( 600  calculations / loop)
Executing ext. loop: 13  of  280  ( 600  calculations / loop)
Executing ext. loop: 14  of  280  ( 600  calculations / loop)
Executing ext. loop: 15  of  280

KeyboardInterrupt: 

In [20]:
from matplotlib import pyplot as plt
%matplotlib widget
plt.figure(figsize=(11, 6))
plt.imshow(out[:,:,5,2].T)#,vmin=6,vmax=9.5) ##alog of density!
#plt.imshow(fenc.T)
#plt.imshow(yobs_a[:,:].T)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
np.set_printoptions(linewidth=180,precision=2,suppress=True)    
out1=np.copy(out)
for i in range(12):
    out1[:,:,i,1]=out1[:,:,i,1]*1e7
    out1[:,:,i,6]=out1[:,:,i,6]*180/np.pi
    out1[:,:,i,7]=out1[:,:,i,7]*180/np.pi
print("    index", "       chi^2","        ne" , "     Y height", "    X los", "     B" ,"          Phi", "      Theta", "      Bx", "        By", "         Bz" )
print(out1[130,270,:,:])

# np.set_printoptions(suppress=False)
# print(out[130,270,:,1])

    index        chi^2         ne      Y height     X los      B           Phi       Theta       Bx         By          Bz
[[4374669.         2.43       8.29       1.13       0.03      18.17      14.08      78.88      17.29       4.34       3.5 ]
 [4381509.         2.43       8.29       1.13       0.03     -18.16     166.93      78.88      17.36      -4.03      -3.5 ]
 [4382781.         2.43       8.29       1.13       0.03     -18.16     195.08     103.15      17.08       4.6        4.13]
 [4389621.         2.43       8.29       1.13       0.03      18.17     347.93     103.15      17.3       -3.7       -4.13]
 [4374759.         3.5        8.29       1.13       0.03      18.34      16.09      78.88      17.29       4.99       3.54]
 [4381419.         3.5        8.29       1.13       0.03     -18.34     164.92      78.88      17.37      -4.68      -3.54]
 [4382871.         3.5        8.29       1.13       0.03     -18.34     197.09     103.15      17.07       5.25       4.17]
 [4389531

In [11]:
print(len(db1[3]),db1[3].shape)
print(db1[3][5,55,43,21,:])

10 (10, 60, 180, 90, 8)
[ 1.00000000e+00 -4.70111299e-03  4.37379255e-02 -4.39165539e-06
  3.88484285e-01 -4.69583030e-05  4.36427498e-04 -3.49742398e-06]


In [7]:
print(len(db2[0]),db2[0].shape)
print(db2[2][5,55,43,21,:])
print(db2[1][5,55,43,21,:])

10 (10, 60, 180, 90, 8)
[ 1.00000000e+00 -4.19970207e-03  3.90317654e-02 -4.32276463e-06
  3.43410422e-01 -4.66130961e-05  4.33219167e-04 -1.53864879e-06]
[ 1.00000000e+00 -3.34808356e-03  3.11496936e-02 -4.35477813e-06
  2.80491796e-01 -2.76344127e-05  2.56832484e-04 -1.25806758e-06]


In [ ]:
gg=cle202_utils.sdbread_par('/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_R0500/fe-xiii_1074/DB0001.DAT',hdrparams,verbose)

READING INDIVIDUAL DATA...
DB file location: /home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_R0500/fe-xiii_1074/DB0001.DAT
77  MB in DB file


In [8]:
print(gg.shape)

print(gg[5,55,43,21,:]/gg[5,55,43,21,0])
for i in range(3,-1,-1):
    gg[:,:,:,:,i]=gg[:,:,:,:,i]/gg[:,:,:,:,0]
print(gg[5,55,43,21,:])


(10, 60, 180, 90, 4)
[ 1.00000000e+00 -4.19970207e-03  3.90317654e-02 -4.32276463e-06]
[ 1.00000000e+00 -4.19970207e-03  3.90317654e-02 -4.32276463e-06]


In [9]:
gg1=cle202_utils.sdbread_par('/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_R0500/fe-xiii_1079/DB0001.DAT',hdrparams,verbose)

READING INDIVIDUAL DATA...
DB file location: /home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_R0500/fe-xiii_1079/DB0001.DAT
77  MB in DB file
13.771253  SECONDS FOR INDIVIDUAL DB READ
----------


In [11]:
print(gg1[5,55,43,21,:])

[ 2.23802118e-07 -3.03779646e-11  2.82330882e-10 -1.00274435e-12]


In [7]:
print(db[2][5,55,43,21,:])

[ 1.00000000e+00 -4.19970207e-03  3.90317654e-02 -4.32276463e-06
  3.43410422e-01 -4.66130961e-05  4.33219167e-04 -1.53864879e-06]


In [13]:

class ctrlparams:
    def __init__(self):
        self.dbdir    = '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_deg_150_fe13/'   ## directory for database
        self.reduce   = 1                                                                                 ## boolean parameter for reduced search
        self.nsearch  = 4                                                                                 ## number of closest solutions to compute
        self.maxchisq = 10                                                                                ## Stop computing solutions above this chi^2 threshold
        self.verbose  = 0                                                                                 ## verbosity parameter
a=ctrlparams()
print(vars(a))

{'dbdir': '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_deg_150_fe13/', 'reduce': 1, 'nsearch': 4, 'maxchisq': 10, 'verbose': 0}
